In [1]:
spark

In [2]:
df = spark.readStream.format('socket').option('host','localhost').option('port',12345).load()

25/10/24 03:58:02 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [3]:
df

DataFrame[value: string]

In [4]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [6]:
# nc -l -p 12345

In [5]:
writer = df.writeStream.format('console').outputMode('Append')

In [6]:
query= writer.start()

25/10/24 03:58:08 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-471d33ec-517c-4ce7-8547-28bee0905184. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/24 03:58:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----------+
|      value|
+-----------+
|How Are You|
+-----------+

-------------------------------------------
Batch: 2
-------------------------------------------
+---------+
|    value|
+---------+
|Iam Happy|
+---------+



In [11]:
query.stop()

In [15]:
writer = df.writeStream.format('console').outputMode('Append').trigger(processingTime='5 seconds')

In [16]:
query = writer.start()

25/10/24 04:00:30 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1c762603-0c2b-458b-8e84-39a465c80504. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/24 04:00:30 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+
|value|
+-----+
|Fares|
|fares|
|fares|
+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+----------------+
|           value|
+----------------+
|Iam Fares Ashraf|
+----------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|THis Batch of Wor...|
+--------------------+



In [17]:
query.stop()

25/10/24 04:01:04 WARN DAGScheduler: Failed to cancel job group 6b96bdb5-25c9-4fe9-8c42-b77881025346. Cannot find active jobs for it.
25/10/24 04:01:04 WARN DAGScheduler: Failed to cancel job group 6b96bdb5-25c9-4fe9-8c42-b77881025346. Cannot find active jobs for it.


In [18]:
from pyspark.sql.functions import split

In [19]:
df_split = df.select(split(df['value'],' ').alias('SplittedLine'))

In [20]:
df_split.printSchema()

root
 |-- SplittedLine: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [21]:
writer = df_split.writeStream.format('console').outputMode('Append').trigger(processingTime='5 seconds')

In [22]:
query = writer.start(truncate= False)

25/10/24 04:01:25 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ad486b8a-4174-4f7d-8ee6-adfa674d6d0f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/24 04:01:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------+
|SplittedLine|
+------------+
+------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+------------+
|SplittedLine|
+------------+
|[Iam, Sad]  |
+------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----------------------------+
|SplittedLine                 |
+-----------------------------+
|[This, is, BY, Fares, Ashraf]|
+-----------------------------+



In [23]:
query.stop()

25/10/24 04:01:54 WARN DAGScheduler: Failed to cancel job group 3ff0b0d7-6a39-46fc-b054-1575a31bd19a. Cannot find active jobs for it.
25/10/24 04:01:54 WARN DAGScheduler: Failed to cancel job group 3ff0b0d7-6a39-46fc-b054-1575a31bd19a. Cannot find active jobs for it.


In [24]:
writer = df_split.writeStream.format('console').outputMode('Append')

In [25]:
query = writer.start()

25/10/24 04:02:03 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-368571dd-80e6-41ab-ad20-be2293f50541. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/24 04:02:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------+
|SplittedLine|
+------------+
+------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+------------+
|SplittedLine|
+------------+
|        [Hi]|
+------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+------------+
|SplittedLine|
+------------+
|[Iam, Fares]|
+------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+
|        SplittedLine|
+--------------------+
|[No, Waiting, Tim...|
+--------------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+------------+
|SplittedLine|
+------------+
|    [Thanks]|
+------------+



In [26]:
query.stop()

25/10/24 04:02:29 WARN DAGScheduler: Failed to cancel job group 204403b8-7f2f-4798-b07f-6f293677cecb. Cannot find active jobs for it.
25/10/24 04:02:29 WARN DAGScheduler: Failed to cancel job group 204403b8-7f2f-4798-b07f-6f293677cecb. Cannot find active jobs for it.


In [27]:
from pyspark.sql.functions import explode

In [28]:
df_words = df_split.select(explode(df_split['SplittedLine']).alias('Words'))

In [29]:
df_counts = df_words.groupBy('words').count()  

In [30]:
df_counts.printSchema()

root
 |-- words: string (nullable = false)
 |-- count: long (nullable = false)



In [31]:
writer = df_counts.writeStream.format('console').outputMode('complete').option('checkpointLocation','ck1')

In [32]:
query = writer.start()

25/10/24 04:02:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
|words|count|
+-----+-----+
+-----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
|words|count|
+-----+-----+
|   by|    1|
|words|    1|
|   Is|    1|
| This|    1|
|GRoup|    1|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
|words|count|
+-----+-----+
|   by|    1|
|  you|    1|
|words|    1|
|  iam|    1|
|   Is|    1|
| meet|    1|
|happy|    1|
| This|    1|
| nice|    1|
|GRoup|    1|
|   to|    1|
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-------+-----+
|  words|count|
+-------+-----+
|     by|    1|
|    you|    1|
| thanks|    1|
|    for|    1|
|  words|    1|
|    iam|    1|
|watcing|    1|
|     Is|    1|
|   meet|    1|
|  happy|    1|
|   This|    1|
|   nice|    1|
|  GRoup|    1|
|     to|    1|
+-------+-----+



In [33]:
query.stop()

25/10/24 04:06:54 WARN DAGScheduler: Failed to cancel job group ccb0f49b-2013-441d-baee-27731cc83a25. Cannot find active jobs for it.
25/10/24 04:06:54 WARN DAGScheduler: Failed to cancel job group ccb0f49b-2013-441d-baee-27731cc83a25. Cannot find active jobs for it.


In [42]:
writer = df_counts.writeStream.format('console').outputMode('complete').option('checkpointLocation','ck1').trigger(processingTime ='1 second')

In [43]:
query = writer.start()

25/10/24 04:13:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
|words|count|
+-----+-----+
+-----+-----+



25/10/24 04:13:59 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000} milliseconds, but spent 17177 milliseconds
25/10/24 04:14:18 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000} milliseconds, but spent 13633 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
|words|count|
+-----+-----+
| Nice|    1|
|  you|    1|
| meet|    1|
|   to|    1|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
|words|count|
+-----+-----+
| Nice|    1|
|  you|    1|
| meet|    1|
|  Iam|    1|
| time|    1|
|     |    1|
|Happy|    1|
|Delay|    1|
|   to|    1|
+-----+-----+



25/10/24 04:14:36 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000} milliseconds, but spent 17453 milliseconds


In [ ]:
# query.stop()

In [3]:
##############################################################################################

In [4]:
from pyspark.sql import SparkSession

In [14]:
spark = SparkSession.builder.getOrCreate()

In [13]:
spark

In [38]:
df =spark.readStream.format('text').load('csv/')

In [39]:
writer = df.writeStream.format('console').outputMode('Append').option('truncate',False)

In [42]:
query = writer.start()

25/10/25 22:40:28 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-cc920f39-29d7-42b7-8067-9a3a15d7bdef. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/25 22:40:28 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------------+
|value                                           |
+------------------------------------------------+
|DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count     |
|United States,Romania,1                         |
|United States,Ireland,264                       |
|United States,India,69                          |
|Egypt,United States,24                          |
|Equatorial Guinea,United States,1               |
|United States,Singapore,25                      |
|United States,Grenada,54                        |
|Costa Rica,United States,477                    |
|Senegal,United States,29                        |
|United States,Marshall Islands,44               |
|Guyana,United States,17                         |
|United States,Sint Maarten,53                   |
|Malta,United States,1                           |
|Bolivia,United States,46           

In [43]:
query.stop()

25/10/25 22:42:37 WARN DAGScheduler: Failed to cancel job group 8d4b474b-ce36-41a8-8f12-5941c7bf7c72. Cannot find active jobs for it.
25/10/25 22:42:37 WARN DAGScheduler: Failed to cancel job group 8d4b474b-ce36-41a8-8f12-5941c7bf7c72. Cannot find active jobs for it.


In [46]:
writer = df.writeStream.format('console').outputMode('Append').option('truncate',False).trigger(processingTime='3 seconds')

In [47]:
query = writer.start()

25/10/25 22:44:20 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6ad3a466-6874-4e6c-a1d6-6897abba751e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/25 22:44:20 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------------+
|value                                           |
+------------------------------------------------+
|DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count     |
|United States,Romania,1                         |
|United States,Ireland,264                       |
|United States,India,69                          |
|Egypt,United States,24                          |
|Equatorial Guinea,United States,1               |
|United States,Singapore,25                      |
|United States,Grenada,54                        |
|Costa Rica,United States,477                    |
|Senegal,United States,29                        |
|United States,Marshall Islands,44               |
|Guyana,United States,17                         |
|United States,Sint Maarten,53                   |
|Malta,United States,1                           |
|Bolivia,United States,46           

In [48]:
query.stop()

25/10/25 22:44:30 WARN DAGScheduler: Failed to cancel job group 19f37c11-0439-43c8-97c2-c8ecd4115442. Cannot find active jobs for it.
25/10/25 22:44:30 WARN DAGScheduler: Failed to cancel job group 19f37c11-0439-43c8-97c2-c8ecd4115442. Cannot find active jobs for it.


In [55]:
#READ FILE AS CSV
from pyspark.sql.functions import *
import pyspark.sql.functions as fn 
from pyspark.sql.types import *


In [59]:
recordSchema = StructType([StructField('date',StringType(),True),
                       StructField('delay',IntegerType(),True),
                       StructField('distance',IntegerType(),True),
                       StructField('origin',StringType(),True),
                       StructField('destination',StringType(),True)])


df = spark.readStream.format('csv').schema(recordSchema).option('header',True).load('csv/csv2')




In [60]:
writer = df.writeStream.format('console').outputMode('Append').option('truncate',False).option('NumRows',30)

In [61]:
query = writer.start()

25/10/25 22:58:17 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-53d95b5e-bf45-4ff8-852c-bd38e7152daf. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/25 22:58:17 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+--------+-----+--------+------+-----------+
|date    |delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|6    |602     |ABE   |ATL        |
|01020600|-8   |369     |ABE   |DTW        |
|01021245|-2   |602     |ABE   |ATL        |
|01020605|-4   |602     |ABE   |ATL        |
|01031245|-4   |602     |ABE   |ATL        |
|01030605|0    |602     |ABE   |ATL        |
|01041243|10   |602     |ABE   |ATL        |
|01040605|28   |602     |ABE   |ATL        |
|01051245|88   |602     |ABE   |ATL        |
|01050605|9    |602     |ABE   |ATL        |
|01061215|-6   |602     |ABE   |ATL        |
|01061725|69   |602     |ABE   |ATL        |
|01061230|0    |369     |ABE   |DTW        |
|01060625|-3   |602     |ABE   |ATL        |
|01070600|0    |369     |ABE   |DTW        |
|01071725|0    |602     |ABE   |ATL        |
|01071230|0    |369     |ABE   |DTW        |
|01

In [62]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [63]:
query.stop()

25/10/25 23:00:30 WARN DAGScheduler: Failed to cancel job group 8a1ad1c0-9d64-46ef-a360-876131b77d22. Cannot find active jobs for it.
25/10/25 23:00:30 WARN DAGScheduler: Failed to cancel job group 8a1ad1c0-9d64-46ef-a360-876131b77d22. Cannot find active jobs for it.


In [75]:
x = spark.read.parquet('parquet/2010-summary.parquet')

In [76]:
x.show()

[Stage 16:>                                                         (0 + 1) / 1]

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [115]:
from pyspark.sql.types import *

recordSchema = StructType([
    StructField('DEST_COUNTRY_NAME', StringType(), False),
    StructField('ORIGIN_COUNTRY_NAME', StringType(), False),
    StructField('count', LongType(), False)
])


df = spark.readStream.format('parquet').schema(recordSchema).load('parquet/2010-summary.parquet/')

writer = df.writeStream.format('console').outputMode('Append').option('truncate',False).option('NumRows',30)

In [116]:
query.stop()

In [117]:
query = writer.start()

25/10/25 23:16:08 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-cd8e48d7-60b3-42ff-a090-4eae9ae51e58. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/25 23:16:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------------------+------------------------------+-----+
|DEST_COUNTRY_NAME               |ORIGIN_COUNTRY_NAME           |count|
+--------------------------------+------------------------------+-----+
|United States                   |Romania                       |1    |
|United States                   |Ireland                       |264  |
|United States                   |India                         |69   |
|Egypt                           |United States                 |24   |
|Equatorial Guinea               |United States                 |1    |
|United States                   |Singapore                     |25   |
|United States                   |Grenada                       |54   |
|Costa Rica                      |United States                 |477  |
|Senegal                         |United States                 |29   |
|United States                   |Marsh

In [118]:
query.stop()

25/10/25 23:17:03 WARN DAGScheduler: Failed to cancel job group ab57fe8b-3817-4792-bd26-4ffbf945f2a1. Cannot find active jobs for it.
25/10/25 23:17:03 WARN DAGScheduler: Failed to cancel job group ab57fe8b-3817-4792-bd26-4ffbf945f2a1. Cannot find active jobs for it.


In [119]:
from pyspark.sql.types import *
import pyspark.sql.functions as fn

recordSchema = StructType([
    StructField('DEST_COUNTRY_NAME', StringType(), False),
    StructField('ORIGIN_COUNTRY_NAME', StringType(), False),
    StructField('count', LongType(), False)
])


df = spark.readStream.format('parquet').schema(recordSchema).load('parquet/2010-summary.parquet/')

df2 = df.groupby('DEST_COUNTRY_NAME').agg(    fn.avg('count').alias('averageCount')   ,   fn.max('count').alias('MaxCount')  )

writer = df2.writeStream.format('console').outputMode('Complete').option('truncate',False).option('NumRows',30)

In [120]:
query = writer.start()

25/10/25 23:20:40 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-acd881f0-15a6-4e2c-93f6-fac75a1f3d7d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/25 23:20:40 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------+------------+--------+
|DEST_COUNTRY_NAME       |averageCount|MaxCount|
+------------------------+------------+--------+
|Paraguay                |90.0        |90      |
|Anguilla                |21.0        |21      |
|Russia                  |152.0       |152     |
|Senegal                 |29.0        |29      |
|Sweden                  |65.0        |65      |
|Kiribati                |17.0        |17      |
|Guyana                  |17.0        |17      |
|Philippines             |132.0       |132     |
|Malaysia                |1.0         |1       |
|Singapore               |25.0        |25      |
|Fiji                    |53.0        |53      |
|Turkey                  |75.0        |75      |
|Germany                 |1392.0      |1392    |
|Afghanistan             |11.0        |11      |
|Jordan                  |50.0        |50      |
|Palau               

In [121]:
query.stop()

25/10/25 23:33:52 WARN DAGScheduler: Failed to cancel job group daae2ae6-009f-4692-9887-7be47531d16b. Cannot find active jobs for it.
25/10/25 23:33:52 WARN DAGScheduler: Failed to cancel job group daae2ae6-009f-4692-9887-7be47531d16b. Cannot find active jobs for it.


In [138]:
# Save Files Only WIth Append Mode Only

from pyspark.sql.types import *
import pyspark.sql.functions as fn

recordSchema = StructType([
    StructField('DEST_COUNTRY_NAME', StringType(), False),
    StructField('ORIGIN_COUNTRY_NAME', StringType(), False),
    StructField('count', LongType(), False)
])


df = spark.readStream.format('parquet').schema(recordSchema).load('parquet/2010-summary.parquet/')

df2 = df.groupby('DEST_COUNTRY_NAME').agg(    fn.avg('count').alias('averageCount')   ,   fn.max('count').alias('MaxCount')  )

writer = df2.writeStream.format('memory').outputMode('Complete').queryName('MyTable')

In [139]:
query = writer.start()

25/10/25 23:44:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a1178590-ca17-4b3f-80d2-6bfd64d73118. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/25 23:44:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

In [143]:
df_sql = spark.sql('select * from MyTable')

In [144]:
df_sql.show()

+--------------------+------------+--------+
|   DEST_COUNTRY_NAME|averageCount|MaxCount|
+--------------------+------------+--------+
|            Paraguay|        90.0|      90|
|            Anguilla|        21.0|      21|
|              Russia|       152.0|     152|
|             Senegal|        29.0|      29|
|              Sweden|        65.0|      65|
|            Kiribati|        17.0|      17|
|              Guyana|        17.0|      17|
|         Philippines|       132.0|     132|
|            Malaysia|         1.0|       1|
|           Singapore|        25.0|      25|
|                Fiji|        53.0|      53|
|              Turkey|        75.0|      75|
|             Germany|      1392.0|    1392|
|         Afghanistan|        11.0|      11|
|              Jordan|        50.0|      50|
|               Palau|        31.0|      31|
|              France|       774.0|     774|
|Turks and Caicos ...|       136.0|     136|
|              Greece|        50.0|      50|
|         

In [145]:
query.stop()

25/10/25 23:47:55 WARN DAGScheduler: Failed to cancel job group 39663778-5255-4694-966a-637c38988d00. Cannot find active jobs for it.
25/10/25 23:47:55 WARN DAGScheduler: Failed to cancel job group 39663778-5255-4694-966a-637c38988d00. Cannot find active jobs for it.


In [149]:
# Save Files Only WIth Append Mode Only

from pyspark.sql.types import *
import pyspark.sql.functions as fn

recordSchema = StructType([
    StructField('DEST_COUNTRY_NAME', StringType(), False),
    StructField('ORIGIN_COUNTRY_NAME', StringType(), False),
    StructField('count', LongType(), False)
])


df = spark.readStream.format('parquet').schema(recordSchema).load('parquet/2010-summary.parquet/')

writer = df.writeStream.format('memory').outputMode('append').queryName('MyTable')

In [150]:
query = writer.start()

25/10/25 23:49:13 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-069ef92b-2578-45b5-86a7-b1f70439f95c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/25 23:49:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [151]:
df_sql = spark.sql('select * from MyTable')

In [152]:
df_sql.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [153]:
query.stop()

25/10/25 23:55:58 WARN DAGScheduler: Failed to cancel job group 04ae4b21-24b7-4970-9d0d-14efd7fcb2d8. Cannot find active jobs for it.
25/10/25 23:55:58 WARN DAGScheduler: Failed to cancel job group 04ae4b21-24b7-4970-9d0d-14efd7fcb2d8. Cannot find active jobs for it.


In [156]:
df = spark.readStream.format('parquet').schema(recordSchema).load('parquet/2010-summary.parquet/')

writer = df.writeStream.format('parquet').outputMode('append').option('checkpointLocation','Stream_chk').option('path','Saved_Stream_Parquet/')

In [157]:
query =writer.start()

25/10/25 23:59:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

In [159]:
query.stop()

25/10/26 00:08:18 WARN DAGScheduler: Failed to cancel job group 80e2ab8c-94b6-412a-9498-673199037c69. Cannot find active jobs for it.
25/10/26 00:08:18 WARN DAGScheduler: Failed to cancel job group 80e2ab8c-94b6-412a-9498-673199037c69. Cannot find active jobs for it.


In [160]:
df.unpersist() # remove from memory

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]